In [1]:
import pygame
import math
import time
import random

pygame.init()

clock = pygame.time.Clock()
fps = 60

#set screen dimension
scrn_width, scrn_height = 1200, 800

#create game window
screen = pygame.display.set_mode((scrn_width, scrn_height))
pygame.display.set_caption('santa_game')

#load background images
bg = pygame.image.load("images/sky.png").convert_alpha() #sky
sky2 = pygame.image.load("images/sky2.png").convert_alpha() #sky
bg_width = bg.get_width()
foreground_1 = pygame.image.load("images/skyline_01.png").convert_alpha()
foreground_2 = pygame.image.load("images/skyline_02.png").convert_alpha()

#load other images
santa = pygame.image.load("images/sleigh1.png").convert_alpha()
prezimage = pygame.image.load("images/present_0.png").convert_alpha()
chiminey_tile = pygame.image.load("images/chimney_base_brick_tile.png").convert_alpha()
chiminey_top_tile = pygame.image.load("images/chimney_top_brick_tile.png").convert_alpha()
moon_img = pygame.image.load("images/moon2.png").convert_alpha()
explosion1 = pygame.image.load("images/explosion_multicooured.png").convert_alpha()
explosion_white = pygame.image.load("images/explosion_white.png").convert_alpha()

#duck
duck = pygame.image.load("images/duck_spritesheet_48px.png").convert_alpha()
duck_animation_frame = 0
duck_framerate = 0
duck_onscreen = []
duck_spawn_counter = 0
explosions = []

#milk
milk_image = pygame.image.load("images/milk.png").convert_alpha()
Milk = []

#font
pygame.font.init()
my_font = pygame.font.SysFont('Impact', 28)

#define game
bg_scroll = 0
sky2_scroll = 0
fg1_scroll = 0
fg2_scroll = 0
scroll_tick = 0
# moon_scroll = 0

tiles = math.ceil(scrn_width / bg_width)
print(tiles)

#create player
dx, dy = 100, 250
player = pygame.Rect(dx, dy, 50, 50)
player_hitbox = pygame.Rect(dx-50, dy, 200, 100)
player_speed = 3
player_speed_diag = math.sqrt((player_speed ** 2)/2)
playertext = 0
playercolor = (255, 0 , 0)

#create chimney
chim_width, chim_height = 100, 300
chimneytext = 0
chimneys = []

chimney_timer = 0
chimney_timer_modifier = 0

chimney_base_speed = 2.5 #pixels per frame
chimney_speed_modifier = 0

chimney_speed_adjusted = 2.5
chimney_gap = random.randrange(4, 7)                 #pixels
chimney_delta = chimney_speed_adjusted / chimney_gap
chimney_spawn_interval = chimney_delta * fps

#create present details
prez_width, prez_height = 30, 30
drop_speed = 5
presents = []
last_drop_time = 0 #tracks last time present was dropped
drop_cooldown = 0.2 #in seconds, cooldown period after drop
presenttext = 0

#create score
score = 0
score_distance = 0
score_presents = 0
score_speed = 0
health = 100
#------------------------------------------------------------------------------------------
#---------------------------------functions------------------------------------------------
#------------------------------------------------------------------------------------------

Santa_Rotation = 0 #change this variable to change the angle santa sits at.
#image rotate function
def Image_Rotation(image, angle): #this code rotates the image around it's center 
    Rotated_Image = pygame.transform.rotate(image, angle)
    return Rotated_Image
#We might want to change what the center of santa is.

#-----------------------Game Over function----------------------------------
final_score = 0
final_score_distance = 0
final_score_presents = 0
final_score_speed = 0
final_score_latch = 0

def End_game_screen():
    global final_score_latch, final_score, final_score_distance, final_score_presents, final_score_speed #because this is a function all global variables have to be defined as such
    global score, score_distance, score_presents, score_speed, health
    global presents, chimneys, duck_onscreen, dx, dy
    text_height = 0
    text_width = 0
    if final_score_latch == 0: #this allows the score to only be recored once.
        final_score = score
        final_score_distance = score_distance
        final_score_presents = score_presents
        final_score_speed = score_speed
        final_score_latch = 1
    screen.fill("black") #this is a very crude option but it just pastes over the entire game so we don't have to remove litterely everything.
    final_scoredisplay = ['Press [Enter] to restart the game', 'Game Over', 'Score: ' + f'{math.floor(final_score)}','Speed: ' + f'{math.floor(final_score_speed)}' + ' mph' , 'Distance: ' + f'{math.floor(final_score_distance)}' + 'm', 'Presents delivered: ' + f'{math.floor(final_score_presents)}'] #each item will be displayed on a new line
    scoretext = []
    for line in final_scoredisplay:
        scoretext.append(my_font.render(line, False, (255, 255, 255)))
    for line in scoretext:
        text_get_height = line.get_height() + 25
        text_get_width = line.get_width()
        text_height -= text_get_height
        #if(text_get_width > text_width):
        text_width = text_get_width
        text_location_Y = ((screen.get_height() + text_height) / 2)
        text_location_X = ((screen.get_width() - text_width) / 2)
        screen.blit(line, (text_location_X, text_location_Y))#moved the score and data to the left, when stuff comes from the right it will be better.
        #text_location_Y += line.get_height() * 5 #line.getheight is the height of the text. 
    for prez in presents: #everything we do want to remove from the screen will need to be removed.
        presents.remove(prez)
    for chim in chimneys:
        chimneys.remove(chim)
    for ducks in duck_onscreen:
        duck_onscreen.remove(ducks)
    for cartons in Milk:
        Milk.remove(cartons)
    dx, dy = 100, 250 #resetting the player back to the "start" location

    if(key[pygame.K_RETURN ]): #when health is above 0 we will drop out of the game over function
        score_distance = 0
        score_presents = 0
        score_speed = 0
        health = 100  
#-----------------------Game Over end----------------------------------

#-----------------------Milk Function----------------------------------
milk_spawn_counter = 0
def Milk_function():
    global milk_spawn_counter, Milk, health, explosions
    milk_spawn_counter += 1
    if(milk_spawn_counter > random.randrange(500, 30000)):
        milk_spawn_counter = 0
        milk_spawn_x0 = 100
        milk_spawn_x1 = screen.get_width() - 100
        Milk.append({"Milk_x": random.randrange(milk_spawn_x0, milk_spawn_x1), 
                     "Milk_y" : 0,
                     "Milk_rotation" : 0, 
                     "Milk_2counter" : 0,
                     "Milk_Rotation_direction" : True})
    for cartons in Milk:
        screen.blit(Image_Rotation(milk_image, cartons["Milk_rotation"]), (cartons["Milk_x"], cartons["Milk_y"]))
        cartons["Milk_y"] += 1
        cartons["Milk_2counter"] += 1
        if(cartons["Milk_2counter"] >= 5):
            cartons["Milk_2counter"] = 0
            if(cartons["Milk_Rotation_direction"] == True):
                cartons["Milk_rotation"] += 1
            else:
                cartons["Milk_rotation"] -= 1
            if(cartons["Milk_rotation"] >= 15):
                cartons["Milk_Rotation_direction"] = False
            if(cartons["Milk_rotation"] <= -15):
                cartons["Milk_Rotation_direction"] = True
        if(cartons["Milk_y"] >= screen.get_height()):
            Milk.remove(cartons)
        if(cartons["Milk_x"]+32 >= player_hitbox.left and cartons["Milk_x"]+32 <= player_hitbox.right and cartons["Milk_y"]+100 >= player_hitbox.top and cartons["Milk_y"]+100 <= player_hitbox.bottom):
            health += 10
            explosions.append({"splode_x" : cartons["Milk_x"]+32,
                               "splode_y" : cartons["Milk_y"]+100,
                               "splode_Animations" : 0, 
                               "splode_framerate" : 0,
                               "splode_animation_Frame" : 0,
                               "splode_image": explosion_white})
            Milk.remove(cartons)
#-----------------------Milk End---------------------------------------
            
#-----------------------Raindeer function------------------------------
Raindeer_image = pygame.image.load("images/Deer_Run_104h.png").convert_alpha()
Raindeer_sprites = []
Raindeer_number = 0
Raindeer_latch = 0
def Raindeer_function():
    global Raindeer_number, Raindeer_sprites, Raindeer_latch
    if(Raindeer_latch == 0):
        Raindeer_number += 1
        if(Raindeer_number <= 3):
            Raindeer_sprites.append({"Raindeer_x" : 120 + Raindeer_number * 120,
                                    "Raindeer_y" : 30,
                                    "Raindeer_Animations" : 0,
                                    "Raindeer_animation_frame" : random.randrange(-1, 5),
                                    "raindeer_framerate" : 0,
                                    "Raindeer_image" : Raindeer_image})
        else:
            Raindeer_latch = 1
    for deer in Raindeer_sprites:
        deer_sprite = deer["Raindeer_image"].subsurface(deer["Raindeer_animation_frame"]*144,0,144,104)
        screen.blit(Image_Rotation(deer_sprite, 0), (dx + deer["Raindeer_x"], dy + deer["Raindeer_y"]))
        deer["raindeer_framerate"] += 1
        if(deer["raindeer_framerate"] >= 10):
            deer["raindeer_framerate"] = 0
            deer["Raindeer_animation_frame"] -= 1
        if(deer["Raindeer_animation_frame"] < 0):
            deer["Raindeer_animation_frame"] = 5
#-----------------------Raindeer end-----------------------------------

#------------------------------------------------------------------------------------------
#---------------------------------functions end--------------------------------------------
#------------------------------------------------------------------------------------------


#game loop
run = True
while run:

    clock.tick(fps)
    #draw scrolling  layers
    for i in range(0, tiles+1):
        screen.blit(bg, (i * bg_width + bg_scroll, 0))
    screen.blit(moon_img, (screen.get_width() - moon_img.get_width() - 50, 50))
    for i in range(0, tiles+1):
        screen.blit(sky2, (i * sky2.get_width() + sky2_scroll, 0))
    for i in range(0, tiles+1):
        screen.blit(foreground_1, (i * foreground_1.get_width() + fg1_scroll, 0))
    for i in range(0, tiles+1):
        screen.blit(foreground_2, (i * foreground_1.get_width() + fg2_scroll, 0))
    # for i in range(0, tiles+1):
    #     screen.blit(moon, (i * moon_width + moon_scroll, 0))

    #scroll background layers
    bg_scroll -= 0.5
    fg1_scroll -= 1
    fg2_scroll -= 1.5
    scroll_tick = 0
    sky2_scroll -= 0.5 

    #reset scroll
    if abs(bg_scroll) > bg_width:
        bg_scroll = 0
    if abs(fg1_scroll) > foreground_1.get_width():
        fg1_scroll = 0
    if abs(fg2_scroll) > foreground_2.get_width():
        fg2_scroll = 0
    if abs(sky2_scroll) > sky2.get_width():
        sky2_scroll = 0
    # if abs(moon_scroll) > moon_width:
    #     moon_scroll = 0

    #txt
    
    #playertext = my_font.render(f'{round(dx)}, {round(dy)}', False, (255, 255, 255))
    #screen.blit(playertext, (0, 0))
    scoredisplay = ['Score: ' + f'{math.floor(score)}','Speed: ' + f'{math.floor(score_speed)}' + ' mph' , 'Distance: ' + f'{math.floor(score_distance)}' + 'm', 'Presents delivered: ' + f'{math.floor(score_presents)}', 'Health: ' + f'{health}'] #each item will be displayed on a new line
    scoretext = []
    for line in scoredisplay:
        scoretext.append(my_font.render(line, False, (255, 255, 255)))
    scoretext_y = 0 #we want to zero this every loop
    for line in scoretext:
        screen.blit(line, (5, scoretext_y))#moved the score and data to the left, when stuff comes from the right it will be better.
        scoretext_y += line.get_height() #line.getheight is the height of the text. 

    #set keyboard actions
    
    #player movement
    key = pygame.key.get_pressed()
    # diag left down
    if ((key[pygame.K_a]) or (key[pygame.K_LEFT])) and ((key[pygame.K_s]) or (key[pygame.K_DOWN])) and ((dx > -25) and (dy < scrn_height-25)):
        player.move_ip(-player_speed_diag, player_speed_diag)
        Santa_Rotation = -5
        dx -= player_speed_diag
        dy += player_speed_diag
    # diag left up
    elif ((key[pygame.K_a]) or (key[pygame.K_LEFT])) and ((key[pygame.K_w]) or (key[pygame.K_UP])) and ((dx > -25) and (dy > -25)):
        player.move_ip(-player_speed_diag, -player_speed_diag)
        Santa_Rotation = 5
        dx -= player_speed_diag
        dy -= player_speed_diag
    # diag right down
    elif ((key[pygame.K_d]) or (key[pygame.K_RIGHT])) and ((key[pygame.K_s]) or (key[pygame.K_DOWN])) and ((dx < scrn_width+10) and (dy < scrn_height-25)):
        player.move_ip(player_speed_diag, player_speed_diag)
        Santa_Rotation = -5
        dx += player_speed_diag
        dy += player_speed_diag
    # diag right up
    elif ((key[pygame.K_d]) or (key[pygame.K_RIGHT])) and ((key[pygame.K_w]) or (key[pygame.K_UP])) and ((dx < scrn_width+10) and (dy > -25)):
        player.move_ip(player_speed_diag, -player_speed_diag)
        Santa_Rotation = 5
        dx += player_speed_diag
        dy -= player_speed_diag
    #left
    elif (key[pygame.K_a] == True) or (key[pygame.K_LEFT]) and (dx > -25):
        Santa_Rotation = 5
        player.move_ip(-player_speed, 0)
        dx -= player_speed
    #right
    elif key[pygame.K_d] or (key[pygame.K_RIGHT]) and (dx < scrn_width+10):
        Santa_Rotation = 0
        player.move_ip(player_speed, 0)
        dx += player_speed
    #up
    elif key[pygame.K_w] or (key[pygame.K_UP]) and (dy > -25):
        Santa_Rotation = 5
        player.move_ip(0, -player_speed)
        dy -= player_speed
    #down
    elif key[pygame.K_s] or (key[pygame.K_DOWN]) and (dy < scrn_height-25):
        Santa_Rotation = -5
        player.move_ip(0, player_speed)
        dy += player_speed
    else:
        Santa_Rotation = 0

    #draw santa at this position
    screen.blit(Image_Rotation(santa, Santa_Rotation), (dx, dy))
    player_hitbox = pygame.Rect(dx+20, dy+25, 200, 100) #recalculate the hit box
    #pygame.draw.rect(screen, playercolor, player_hitbox) #display the hit box, comment out to unsee
    Raindeer_function()

    # if you get a present in the chimney
    for prez in presents:
        #get coordinates for present corners
        present_coords = {"topleft": [prez["x"], prez["y"]],
                          "topright": [prez["x"]+prez_width, prez["y"]],
                          "bottomleft": [prez["x"], prez["y"]+prez_height],
                          "bottomright": [prez["x"]+prez_width, prez["y"]+prez_height]}
        for chim in chimneys:
            #get coords for chimney corners
            chimney_coords = {"topleft": [chim["x"], chim["y"]],
                          "topright": [chim["x"]+chim['cw'], chim["y"]],
                          "bottomleft": [chim["x"], chim["y"]+chim['ch']],
                          "bottomright": [chim["x"]+chim['cw'], chim["y"]+chim['ch']]}
            #scoring
            corner_count = 0
            for coord in present_coords.values():
                if (coord[0] >= chimney_coords["topleft"][0]) and (coord[0] <= chimney_coords["topright"][0]) and (coord[1] >= chimney_coords["topleft"][1]):
                    corner_count += 1
            if corner_count >= 4:
                score_presents += 1
                presents.remove(prez)
            corner_count = 0

            #right collision (hit right side of present/left of chimney)
            if (present_coords["topright"][0] > chimney_coords['topleft'][0]) and (
                    present_coords['topright'][0] < chimney_coords['topleft'][0]+10) and (
                    present_coords["topright"][1] >= chimney_coords["topleft"][1]):
                prez["speedx"] = chimney_base_speed * 2
                prez["Rot_direction"] = not prez["Rot_direction"]
            # left collision (hit left side of present/right of chimney)
            if (present_coords["topleft"][0] > chimney_coords['topright'][0]-10) and (
                    present_coords['topleft'][0] < chimney_coords['topright'][0]) and (
                    present_coords["topleft"][1] >= chimney_coords["topright"][1]):
                prez["speedx"] = chimney_base_speed * -0.5
                prez["Rot_direction"] = not prez["Rot_direction"]
            #bottom right collision (bottom right cornre of prz bounce off top of chim)
            if (present_coords["bottomright"][0] > chimney_coords['topleft'][0]) and (
                    present_coords['bottomright'][0] < chimney_coords['topleft'][0]+20) and (
                    present_coords["bottomright"][1] < chimney_coords["topleft"][1]+20) and (
                    present_coords["bottomright"][1] > chimney_coords["topleft"][1]
            ):
                prez["speedx"] = chimney_base_speed * 2
                prez["speedy"] *= -1
                #prez["speedy"] -= 10
                prez["Rot_direction"] = not prez["Rot_direction"]

            # bottom left collision (bottom left cornre of prz bounce off top of chim)
            if (present_coords["bottomleft"][0] > chimney_coords['topright'][0]-20) and (
                    present_coords['bottomleft'][0] < chimney_coords['topright'][0]) and (
                    present_coords["bottomleft"][1] < chimney_coords["topright"][1]) and (
                    present_coords["bottomleft"][1] > chimney_coords["topright"][1]-20
            ):
                prez["speedx"] = chimney_base_speed * -0.5
                prez["speedy"] *= -1
                prez["Rot_direction"] = not prez["Rot_direction"]

                #prez["speedy"] -= 10

            #any prez corner hits outside wall of chim, bounce out
            #when we're 90% inside the top boundaries, it should score
            #should bounce upwards if hits top and doen't go in
            #should bounce sideways and down if hits side

    #drop present with spacebar
    current_time = time.time()
    if key[pygame.K_SPACE] & (current_time - last_drop_time > drop_cooldown):
        #add present & its coordinates to list of presents
        presents.append({"x": dx+80, 
                         "y": dy+20, 
                         "speedx": 0, 
                         "speedy": 1, 
                         "Rotation": 0, 
                         "Rot_direction": True}) #add present coordinates to list
        last_drop_time = current_time #reset last drop so cooldown can be in effect
    # set present movement
    for prez in presents:
        prez["speedy"] += 0.4
        prez["y"] += prez["speedy"] #add movement speed to coordinate so it moves
        prez["x"] -= prez["speedx"]
        if prez["Rot_direction"]:
            prez["Rotation"] -= 1
        else:
            prez["Rotation"] += 1

        if prez["y"] > scrn_height:
            presents.remove(prez)
        screen.blit(Image_Rotation(prezimage, prez["Rotation"]), (prez["x"]-5, prez["y"]-5))
    
    #spawn chimney
    #need to update chimney timer so that chimneys get spawned faster as score increases, otherwise they appear to be closer togethr
    chimney_timer += 1
    if chimney_timer >= chimney_spawn_interval:
        chimney_height = random.randrange(200, 400)
        chimney_width = random.randrange(80, 150)
        chimneys.append({"ch": chimney_height,
                         "cw": chimney_width,
                         "x": scrn_width,
                         "y": scrn_height-chimney_height})
        chimney_timer = 0
        chimney_gap = random.randrange(3, 9)

    chimney_delta = chimney_gap / chimney_speed_adjusted
    chimney_spawn_interval = chimney_delta * fps

    #move chimney
    for chim in chimneys:
        #gradually increase chimney movement speed
        #every 10 points gives a speed increase of .5
        chimney_speed_modifier = math.floor(score_presents/10)*0.5
        chimney_speed_adjusted = chimney_base_speed + chimney_speed_modifier
        chim["x"] -= chimney_speed_adjusted
        if chim["x"] < -chim["cw"]:
            chimneys.remove(chim)
        chim_tile_width = chim["cw"] / 32
        chim_tile_height = chim["ch"] / 32
        for y in range(math.ceil(chim_tile_height)):
            for x in range(math.ceil(chim_tile_width)):
                if (chim["cw"]-(x*32)) < 32:
                    cropped_chim = chiminey_tile.subsurface(0,0,chim["cw"]-(x*32),32)
                    screen.blit(cropped_chim, (chim["x"] + x*32, chim["y"] + y*32))
                else:
                    screen.blit(chiminey_tile, (chim["x"] + x*32, chim["y"] + y*32))

    #duck
    duck_spawn_counter += 1 
    if (duck_spawn_counter > random.randrange(200, 1000)):
        duck_spawn_counter = 0
        duck_x_spawn = screen.get_width()
        duck_y_spawn = random.randrange(10, 600)
        duck_onscreen.append({"Duck_x" : duck_x_spawn,
                              "Duck_y" : duck_y_spawn, 
                              "Duck_Animations" : 0, 
                              "Duck_framerate" : 0,
                              "Duck_animation_Frame" : 0})
               
    for ducks in duck_onscreen:
        ducks["Duck_x"] -= 1 
        duck_sprite = duck.subsurface(0, ducks["Duck_animation_Frame"]*48,48,48)
        screen.blit(duck_sprite, (ducks["Duck_x"], ducks["Duck_y"]))
        ducks["Duck_framerate"] += 1
        if(ducks["Duck_framerate"] >= 8):
            ducks["Duck_x"] -= 1
            ducks["Duck_framerate"] = 0
            ducks["Duck_animation_Frame"] += 1
        if(ducks["Duck_animation_Frame"] >= 4):
            ducks["Duck_animation_Frame"] = 0
        if(ducks["Duck_x"] < -48):
            duck_onscreen.remove(ducks)
        if(ducks["Duck_x"] >= player_hitbox.left and ducks["Duck_x"] <= player_hitbox.right and ducks["Duck_y"] >= player_hitbox.top and ducks["Duck_y"] <= player_hitbox.bottom):
            health -= 10
            #explosion1
            explosions.append({"splode_x" : ducks["Duck_x"],
                               "splode_y" : ducks["Duck_y"],
                               "splode_Animations" : 0, 
                               "splode_framerate" : 0,
                               "splode_animation_Frame" : 0,
                               "splode_image": explosion1})
            duck_onscreen.remove(ducks)

    for splodes in explosions:
        splode_sprite = splodes["splode_image"].subsurface((splodes["splode_animation_Frame"]*33), 0, 33, 33)
        screen.blit(splode_sprite, (splodes["splode_x"], splodes["splode_y"]))
        splodes["splode_framerate"] += 1
        if(splodes["splode_framerate"] >= 4):
            splodes["splode_framerate"] = 0
            splodes["splode_animation_Frame"] += 1
        if(splodes["splode_animation_Frame"] >= 5):
            explosions.remove(splodes)

    Milk_function()
          
    if(health <= 0):
        End_game_screen()

    #score calcuations
    score_distance += chimney_speed_adjusted / 10 #trying to get the scaling to feel right, /10 seems ok if the distnce is in meters
    score_speed = chimney_speed_adjusted * 4 #scaling again, *4 seems ok for MPH
    score = (score_presents * chimney_speed_adjusted * 4) + (score_distance / 10) #this is the calculation for the score, I math.floor it when we display it, but it will be the full float here.

    #event handler
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False

    pygame.display.update()

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
2
